In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#change the subpath to match your drive location
subpath = '/content/drive/MyDrive/2020-Move/Learning'
path = subpath+'/DS4A-correlation-one/DS4A Project/repo/data/'
# We can combine the datasets into one DF here and begin our analyses

In [4]:
import pandas as pd
import numpy as np
!pip install datetime-quarter
from datequarter import DateQuarter

Names of files to be merged:

Wildfires:
fires_by_county.csv and fires_by_zip.csv

Airports:
airport_data_aggregates.csv

Airbnb:
Availability_by_City.csv,
Availability_by_Zipcode.csv,
(to be added) Availability_by_County.csv

AQI:
BA_AQI_counties.csv


Columns:

Date (datetime) and county or zipcode used to merge Airbnb, AQI, and wildfires

Date (quarters) can be used to append airport data after the prior 3 datasets

In [5]:
#drive path
path = '/content/drive/MyDrive/2020-Move/Learning/DS4A-correlation-one/DS4A Project/repo/data/'

# reading all of the datasets from the data folder
df_airbnb_city = pd.read_csv(path+'Complete_Availabitlity_by_City.csv') #no county breakdown, can merge for EDA using zipcodes file
df_airport = pd.read_csv(path+'airport_data_aggregates.csv') #intentionally no breakdown, can merge by quarter 

df_airbnb_county = pd.read_csv(path+'Complete_Availabitlity_by_County.csv')
df_aqi_county = pd.read_csv(path+'BA_AQI_COUNTIES.csv') #merged
df_fires_county = pd.read_csv(path+'fires_by_county.csv') #merged
df_mobility = pd.read_csv(path+'BA_County_Mobility_2020.csv') #merged


df_airbnb_zip = pd.read_csv(path+'Complete_Availabitlity_by_Zipcode.csv') #merged-zip
df_aqi_zip = pd.read_csv(path+'BA_AQI_ZIP_CODES.csv') #merged-zip
df_fires_zip = pd.read_csv(path+'fires_by_zip.csv') #merged-zip

In [6]:
def see_df(df):
  print('Data dimensions:', df.shape, '\n')
  return df.head(2)

In [7]:
see_df(df_airbnb_city)

Data dimensions: (26433, 3) 



,City,Date,Value
0,Atherton,2020-06-26,0.033333
1,Atherton,2020-06-27,0.229885


In [8]:
df_airbnb_city['City'].unique() #how to merge

array(['Atherton', 'Belmont', 'Berkeley', 'Brisbane', 'Burlingame',
       'Campbell', 'Cupertino', 'Daly City', 'Emeryville', 'Fremont',
       'Gilroy', 'Half Moon Bay', 'Hayward', 'Los Altos', 'Los Gatos',
       'Milbrae', 'Milpitas', 'Morgan Hill', 'Mountain View', 'Oakland',
       'Pacifica', 'Palo Alto', 'Pescadero', 'Portola Valley',
       'Redwood City', 'San Bruno', 'San Carlos', 'San Francisco',
       'San Jose', 'San Mateo', 'Santa Clara', 'Saratoga', 'Sunnyvale'],
      dtype=object)

In [9]:
see_df(df_airbnb_zip)

Data dimensions: (187747, 3) 



,Zipcode,Date,Value
0,90103,2015-05-04,0.190083
1,90103,2015-05-05,0.223529


In [22]:
see_df(df_airbnb_county)

Data dimensions: (6770, 3) 



,County,Date,Value
0,Alameda,2015-06-29,0.142857
1,Alameda,2015-06-30,0.157303


In [10]:
see_df(df_airport)

Data dimensions: (33564, 8) 



,QQ-YYYY,PASSENGERS,MEDIAN_ITIN_FARE,AVG_ITIN_FARE,MIN_ITIN_FARE,MAX_ITIN_FARE,ORIGIN_CITY_NAME,DEST_CITY_NAME
0,Q4 of 2010,1.0,10.0,10.000000,10.0,10.0,"Allentown/Bethlehem/Easton, PA","Oakland, CA"
1,Q4 of 2010,59.0,466.0,457.350877,10.0,1059.0,"Allentown/Bethlehem/Easton, PA","San Francisco, CA"


In [11]:
see_df(df_fires_county)

Data dimensions: (17440, 4) 



,Unnamed: 0,Date,Active_wildfires,County
0,0,2015-01-01,False,Alameda
1,1,2015-01-02,False,Alameda


In [12]:
see_df(df_fires_zip)

Data dimensions: (664900, 4) 



,Unnamed: 0,Date,Active_wildfires,Zipcode
0,0,2015-01-01,False,94002
1,1,2015-01-02,False,94002


In [13]:
see_df(df_mobility)

Data dimensions: (2916, 8) 



,COUNTY,DATE,RETAIL/RECREATION,GROCERY/PHARMACY,PARKS,TRANSIT/STATIONS,WORKPLACES,RESIDENTIAL
0,Alameda,2020-02-15,1.0,0.0,22.0,1.0,0.0,-1.0
1,Alameda,2020-02-16,7.0,0.0,24.0,5.0,1.0,-2.0


In [14]:
see_df(df_aqi_county)

Data dimensions: (19728, 8) 



,DATE,COUNTY,24hr_AQI,AQI_LEVEL,AQI_LEVEL_MAX,AVE_PM_2.5,MAX_PM_2.5,Max_AQI
0,2015-01-01,Alameda,68.0,2.0,3.0,19.962676,52.0,142.0
1,2015-01-01,Contra Costa,71.0,2.0,4.0,21.403465,66.0,156.0


In [15]:
see_df(df_aqi_zip)

Data dimensions: (668560, 8) 



,Zip Code,DATE,AVE_PM_2.5,MAX_PM_2.5,24hr_AQI,Max_AQI,STATION,ESTIMATED
0,94002,2015-01-01,14.708333,40.0,56,112,Redwood City,False
1,94002,2015-01-02,15.500000,25.0,58,78,Redwood City,False


In [17]:
#starting with AQI and county
main_df_county = df_aqi_county.merge(df_fires_county, how='left', left_on=['COUNTY', 'DATE'], right_on=['County', 'Date'])
main_df_county = main_df_county.drop(columns=['County','Date', 'Unnamed: 0'])
see_df(main_df_county)

Data dimensions: (19728, 9) 



,DATE,COUNTY,24hr_AQI,AQI_LEVEL,AQI_LEVEL_MAX,AVE_PM_2.5,MAX_PM_2.5,Max_AQI,Active_wildfires
0,2015-01-01,Alameda,68.0,2.0,3.0,19.962676,52.0,142.0,False
1,2015-01-01,Contra Costa,71.0,2.0,4.0,21.403465,66.0,156.0,False


In [20]:
#adding mobility data
main_df_county_2 = main_df_county.merge(df_mobility, how='left', on=['COUNTY', 'DATE'])
see_df(main_df_county_2)

Data dimensions: (19728, 15) 



,DATE,COUNTY,24hr_AQI,AQI_LEVEL,AQI_LEVEL_MAX,AVE_PM_2.5,MAX_PM_2.5,Max_AQI,Active_wildfires,RETAIL/RECREATION,GROCERY/PHARMACY,PARKS,TRANSIT/STATIONS,WORKPLACES,RESIDENTIAL
0,2015-01-01,Alameda,68.0,2.0,3.0,19.962676,52.0,142.0,False,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-01,Contra Costa,71.0,2.0,4.0,21.403465,66.0,156.0,False,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
#adding airbnb data
main_df_county_3 = main_df_county_2.merge(df_airbnb_county, how='left', left_on=['COUNTY', 'DATE'], right_on=['County', 'Date'])

#prettify columns
main_df_county_3 = main_df_county_3.drop(columns=['County', 'Date']).rename(columns={'Value':'Vacancy_Rate'})
see_df(main_df_county_3)

Data dimensions: (19728, 16) 



,DATE,COUNTY,24hr_AQI,AQI_LEVEL,AQI_LEVEL_MAX,AVE_PM_2.5,MAX_PM_2.5,Max_AQI,Active_wildfires,RETAIL/RECREATION,GROCERY/PHARMACY,PARKS,TRANSIT/STATIONS,WORKPLACES,RESIDENTIAL,Vacancy_Rate
0,2015-01-01,Alameda,68.0,2.0,3.0,19.962676,52.0,142.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-01,Contra Costa,71.0,2.0,4.0,21.403465,66.0,156.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
#To do a zip breakdown:
print('aqi size', df_aqi_zip.shape)
print('fires size', df_fires_zip.shape)
print('airbnb size', df_airbnb_zip.shape)

aqi size (668560, 6)
fires size (664900, 4)
airbnb size (187747, 3)


In [30]:
main_df_zip = df_aqi_zip.merge(df_fires_zip, how='left', left_on=['Zip Code', 'DATE'], right_on=['Zipcode', 'Date'])
main_df_zip_2 = main_df_zip.merge(df_airbnb_zip, how='left', left_on=['Zip Code', 'DATE'], right_on=['Zipcode', 'Date'])

#prettify columns
main_df_zip_2 = main_df_zip_2.drop(columns=['Unnamed: 0', 'Date_x','Zipcode_x', 'Zipcode_y', 'Date_y'])
main_df_zip_2 = main_df_zip_2.rename(columns={'Value':'Vacancy_Rate'})

print(main_df_zip_2.shape)
main_df_zip_2.head(2)

(668560, 8)


,Zip Code,DATE,AVE_PM_2.5,MAX_PM_2.5,24hr_AQI,Max_AQI,Active_wildfires,Vacancy_Rate
0,94002,2015-01-01,14.708333,40.0,56,112,False,NaN
1,94002,2015-01-02,15.500000,25.0,58,78,False,NaN


In [31]:
#merge county for which the zipcode belongs, county table can refer to that
zipcodes = pd.read_json(path+'BA_ZIP_CODES.json').transpose()
zipcodes.head(2)

,ZIP_CODE,POPULATION_2010,HOUSING_2010,LAND_AREA_SQ_METER,LAND_AREA_SQ_MILES,LATITUDE,LONGITUDE,COUNTY
94002,94002,25992,11108,14668197,5.663,37.5144,-122.299,San Mateo
94005,94005,4282,1934,11384384,4.396,37.6888,-122.409,San Mateo


In [37]:
#merge for real - add counties so we can reference later if we need to
df_zipcodes_final = main_df_zip_2.merge(zipcodes[['ZIP_CODE','COUNTY']], how='left', left_on='Zip Code', right_on='ZIP_CODE')
#quarter field for zip table
func_dt = lambda x, y: DateQuarter(x, y)
df_zipcodes_final['DATE'] = pd.to_datetime(df_zipcodes_final['DATE'])
df_zipcodes_final['quarter'], df_zipcodes_final['year'] =  df_zipcodes_final['DATE'].dt.quarter, df_zipcodes_final['DATE'].dt.year
df_zipcodes_final['QUARTER'] = df_zipcodes_final[['year', 'quarter']].apply(lambda x: func_dt(*x), axis=1)
df_zipcodes_final = df_zipcodes_final.drop(columns=['year', 'quarter','ZIP_CODE'])
see_df(df_zipcodes_final)

Data dimensions: (668560, 10) 



,Zip Code,DATE,AVE_PM_2.5,MAX_PM_2.5,24hr_AQI,Max_AQI,Active_wildfires,Vacancy_Rate,COUNTY,QUARTER
0,94002,2015-01-01,14.708333,40.0,56,112,False,NaN,San Mateo,Q1 of 2015
1,94002,2015-01-02,15.500000,25.0,58,78,False,NaN,San Mateo,Q1 of 2015


In [38]:
#here listing final things
df_counties_final = main_df_county_3
df_airports_final = df_airport

In [39]:
see_df(df_counties_final)

Data dimensions: (19728, 16) 



,DATE,COUNTY,24hr_AQI,AQI_LEVEL,AQI_LEVEL_MAX,AVE_PM_2.5,MAX_PM_2.5,Max_AQI,Active_wildfires,RETAIL/RECREATION,GROCERY/PHARMACY,PARKS,TRANSIT/STATIONS,WORKPLACES,RESIDENTIAL,Vacancy_Rate
0,2015-01-01,Alameda,68.0,2.0,3.0,19.962676,52.0,142.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-01,Contra Costa,71.0,2.0,4.0,21.403465,66.0,156.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
#save to files
df_zipcodes_final.to_csv(path+'aggregated_by_zip.csv', index=False)
df_counties_final.to_csv(path+'aggregated_by_county.csv', index=False)
df_airports_final.to_csv(path+'aggregated_by_departure.csv', index=False)